# Hanks

Il verbo transitivo che ho scelto è il verbo _raise_. Le frasi che compongono il corpus sono memorizzate nel file `sentences_raise.txt` e sono state prese da vari siti web.

## Import

In [114]:
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import spacy
from spacy.matcher import DependencyMatcher
from collections import Counter

## Variabili globali

Utilizziamo una variabile `special_cases_dict` che permette di assegnare il nome di un synset a una serie di parole speciali, come i pronomi, che non sono presenti in wordnet. Le parole sono rappresentate dalla coppia (tag, parola) che permette una migliore disambiguazione.

In [115]:
special_cases_dict = {
        'act.n.02': [('TO', 'to')],
        'entity.n.01': [('DT', 'what'), ('WP', 'what'), ('NN', 'something'), ('NN', 'anything'), ('NN', 'everything'), 
                        ('DT', 'this'), ('WDT', 'this'), ('DT', 'these'), ('WDT', 'that'), ('DT', 'that'), ('DT', 'those')],
        'person.n.01': [('PRP', 'i'), ('PRP', 'you'), ('PRP', 'he'), ('PRP', 'she'), 
                        ('PRP', 'we'), ('PRP', 'they'), ('PRP', 'him'), ('PRP', 'her'), 
                        ('PRP', 'them'), ('PRP', 'herself'), ('PRP', 'himself'), ('NNP', 'proper_name'),
                        ('NN', 'anyone'), ('NN', 'proper_name'), ('PRP', 'themselves'), ('WP', 'whom')  ],
        'animal.n.01': [('PRP', 'it'), ('PRP', 'itself'), ('NNS', 'koi')],
        'artifact.n.01': [('WDT', 'which')],
        'food.n.01': [('NNP', 'kosra')],
        'number.n.02': [('CD', 'number')]
    }

Per il parsing delle frasi ho utilizzato il dependecyMatcher di spacy. Ho selezionato 3 pattern che permettono di riconoscere soggetto e oggetto del verbo. Ogni pattern è corroborato da una frase di esempio di quel pattern.

C'è un quarto pattern che ho implementato e provato ad usare, ma questo causa problemi nell'estrazione di oggetto e soggetto del verbo da un match del dependency matcher ed inoltre si attiva per circa lo 0.6% delle frasi del corpus, e quindi è poco rilevante. I problemi derivano dal fatto che il quarto pattern abbia tipicamente soggetto e oggetto entrambi prima del verbo raise. L'estrazione di oggetto e soggetto da un match si aspetta, invece, che la prima parola del match sia il soggetto e l'ultima l'oggetto.

In [116]:
nlp = spacy.load('en_core_web_sm')
matcher = DependencyMatcher(nlp.vocab)

# Example: I raised you better than this!
# Pattern: raised --nsubj--> I
#                 --pobj/dobj--> you
pattern1 = [
  {
    "RIGHT_ID": "anchor_raise",
    "RIGHT_ATTRS": {"LEMMA": "raise"}
  },
  {
    "LEFT_ID": "anchor_raise",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": "nsubj"}
  },
  {
    "LEFT_ID": "anchor_raise",
    "REL_OP": ">",
    "RIGHT_ID": "obj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["pobj", "dobj"]}}
  },
]

# Example: He was raised by his grandparents.
# Pattern: raised --nsubjpass--> He
#                 --prep/agent--> by --pobj--> grandparents
pattern2 = [
  {
    "RIGHT_ID": "anchor_raise",
    "RIGHT_ATTRS": {"LEMMA": "raise"}
  },
  {
    "LEFT_ID": "anchor_raise",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": "nsubjpass"}
  },
  {
    "LEFT_ID": "anchor_raise",
    "REL_OP": ">",
    "RIGHT_ID": "prep_or_agent",
    "RIGHT_ATTRS": {"DEP": {"IN": ["prep", "agent"]}}
  },
  {
    "LEFT_ID": "prep_or_agent",
    "REL_OP": ">",
    "RIGHT_ID": "obj",
    "RIGHT_ATTRS": {"DEP": "pobj"}
  },
]

# Example: You need to raise your hand.
# Pattern: need --?--> raise --dobj--> hand
#               --nsubj--> you
pattern3 = [
  {
    "RIGHT_ID": "anchor_verb",
    "RIGHT_ATTRS": {"POS": "VERB"}
  },
  {
    "LEFT_ID": "anchor_verb",
    "REL_OP": ">",
    "RIGHT_ID": "raise",
    "RIGHT_ATTRS": {"LEMMA": "raise"}
  },
  {
    "LEFT_ID": "anchor_verb",
    "REL_OP": ">",
    "RIGHT_ID": "nsubj",
    "RIGHT_ATTRS": {"DEP": "nsubj"}
  },
  {
    "LEFT_ID": "raise",
    "REL_OP": ">",
    "RIGHT_ID": "obj",
    "RIGHT_ATTRS": {"DEP": "dobj"}
  }
]

# PATTERN COMMENTATO PERCHE' CAUSA PROBLEMI NELL'ESTRAZIONE DI SUBJ E OBJ ED E' UTILIZZATO SU POCHE FRASI
# # Example: The money they raised were used to buy some books.
# # Pattern: raised --nsubj--> they
# #                 --?--> money
# pattern4 = [
#   {
#     "RIGHT_ID": "anchor_raise",
#     "RIGHT_ATTRS": {"LEMMA": "raise"}
#   },
#   {
#     "LEFT_ID": "anchor_raise",
#     "REL_OP": ">",
#     "RIGHT_ID": "subj",
#     "RIGHT_ATTRS": {"DEP": "nsubj"}
#   },
#   {
#     "LEFT_ID": "anchor_raise",
#     "REL_OP": "<",
#     "RIGHT_ID": "obj",
#     "RIGHT_ATTRS": {"POS": {"IN": ["NOUN", "PRON"]}}
#   },
# ]

matcher.add("pattern1", [pattern1])
matcher.add("pattern2", [pattern2])
matcher.add("pattern3", [pattern3])
# matcher.add("pattern4", [pattern4])

## Recupero delle frasi

In [117]:
def retrieve_sentences(read_path):
    sentences = []
    with open(read_path, mode='r', encoding='utf8') as f:
        for sent in f.readlines():
            sentences.append(sent.strip('\n'))
    return sentences, len(sentences)
    
path = 'resources/sentences_raise.txt'
corpus, n = retrieve_sentences(path)
print(f'Number of sentences with verb "raise" is: {n}')

Number of sentences with verb "raise" is: 693


## Processing

A questo punto processiamo il corpus di frasi per cercare i pattern che abbiamo definito prima, estraendo oggetto e soggetto del verbo raise.

In [118]:
def process_corpus(corpus):
    result = list()
    for i, sent in enumerate(corpus):
        subj, obj, subj_tag, obj_tag = get_subj_and_obj(sent)
        if subj == '' or obj == '':
            continue
        subj_sense, obj_sense = retrieve_supersenses(subj, obj, subj_tag, obj_tag, sent)
        tmp = {
            'subj': subj,
            'subj_sense': subj_sense,
            'obj': obj,
            'obj_sense': obj_sense
        }
        result.append(tmp)
    return result
        
def get_subj_and_obj(sent):
    sent = nlp(sent)
    matches = matcher(sent)
    for match in matches:
        match_words = sorted(match[1])
        phrase = sent[match_words[0]:match_words[len(match_words)-1]+1]
        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text
        
        return subj,dobj,phrase[0].tag_,phrase[len(phrase)-1].tag_
    return '', '', '', ''

def retrieve_supersenses(subj, obj, subj_tag, obj_tag, sent):
    words = [subj, obj]
    tags = [subj_tag, obj_tag]
    senses = []
    for i in range(len(words)):
        sense = get_sense(words[i], tags[i], sent)
        if sense is None:
            senses.append('unknown')
        elif sense.lexname() == 'noun.Tops' and sense.name() != 'entity.n.01':
            w, _, _ = sense.name().split('.')
            senses.append("noun." + w)
        else:
            senses.append(sense.lexname())
    return senses

def get_sense(word, tag, sent):
    special = special_cases(word, tag)
    if special is not None:
        return special
    return lesk(sent, word)

def special_cases(word, tag):
    if tag in ['NNP', 'NN'] and word[0].isupper():
        word = 'proper_name'
    if tag == 'CD':
        word = 'number'
    for k, v in special_cases_dict.items():
        if (tag, word.lower()) in v:
            return wn.synset(k)
    return None

obj_subj_list = process_corpus(corpus)

## Risultati

Nell'esposizione dei risultati la prima cosa da verificare è quante frasi i pattern utilizzati siano stati in grado di catturare. Vediamo quindi che la percentuale è poco superiore al 50%. Aggiungendo altri pattern è possibile aumentarla. Questo però richiederebbe un lavoro lungo e poco utile ai fini dell'esercizio, pertanto si è scelto di limitarsi ai tre pattern sopra presentati.

In [119]:
def aggregate_results(obj_subj_list):
    couples = []
    for sent in obj_subj_list:
        tmp = (sent['subj_sense'], sent['obj_sense'])
        couples.append(tmp)
    counter = Counter(couples)
    sorted_counter = dict(sorted(counter.items(), key=lambda x: x[1], reverse=True))
    return sorted_counter

def count_used_sentences(counter):
    sum = 0
    for v in counter.values():
        sum += v
    return sum

counter = aggregate_results(obj_subj_list)
l = count_used_sentences(counter)
perc_sent_used = round(l/n*100, 2)
print(f'Percentuale di frasi nel corpus che sono state processate: {perc_sent_used}%')

Percentuale di frasi nel corpus che sono state processate: 53.82%


A questo punto analizziamo i risultati che abbiamo ottenuto, cercando di ignorare quelli meno rilevanti.

In [121]:
def delete_less_frequent(counter, threshold):
    new_counter = counter.copy()
    count = 0
    for key, value in counter.items():
        if value < threshold:
            count += 1
            del new_counter[key]
    return new_counter, count

threshold = 4
results, n_deleted = delete_less_frequent(counter, threshold)
print(f'Of {len(counter)} total pairs, the {n_deleted} who occured in less that {threshold} different sentences were deleted')
results

Of 147 total pairs, the 129 who occured in less that 4 different sentences were deleted


{('noun.person', 'noun.person'): 52,
 ('noun.person', 'noun.possession'): 24,
 ('noun.person', 'noun.artifact'): 19,
 ('noun.person', 'noun.cognition'): 16,
 ('noun.person', 'noun.act'): 16,
 ('noun.person', 'verb.competition'): 15,
 ('noun.person', 'verb.communication'): 8,
 ('noun.person', 'noun.group'): 8,
 ('noun.person', 'noun.quantity'): 7,
 ('noun.person', 'noun.attribute'): 7,
 ('noun.person', 'noun.animal'): 7,
 ('noun.person', 'noun.communication'): 6,
 ('noun.communication', 'verb.communication'): 4,
 ('noun.group', 'noun.communication'): 4,
 ('noun.person', 'noun.relation'): 4,
 ('noun.person', 'adj.all'): 4,
 ('noun.artifact', 'noun.person'): 4,
 ('noun.person', 'verb.contact'): 4}

Nell'analizzare i risultati è evidente come la grande maggioranza delle volte il soggetto accettato dal verbo "raise" è una persona, o un gruppo di individui. Il complemento oggetto, invece può variare in modo molto più libero tra i vari supersensi di WordNet.

Vediamo che la prima coppia di supersensi è (noun.person, noun.person). Questo corrisponde all'utilizzo del termine raise con il significato di _crescere/allevare/educare_, come per esempio _"He raised that child well"_. La seconda, invece, è (noun.person, noun.possession) che corrisponde al significato di _fare una raccolta fondi_, come nella frase _"The students are raising money for their school"_. La terza coppia, invece, è (noun.person, noun.artifact) che corrisponde al significato di _sollevare_ all'utilizzo in frasi come _"He raised the torch in order to better see in the darkness"_. La quarta coppia, (noun.person, noun.cognition), corrisponde al significato di _fare una domanda_/_portare all'attenzione_ come nella frase _"He raised a question"_.

E' interessante notare come per la prima coppia il significato espresso poc'anzi non sia l'unico: per esempio nell'ambito della danza, avrebbe senso dire _"He raised her over his shoulders"_, dove il verbo _raise_ non ha il significato di _crescere_, ma di _sollevare_.